# AlexNet 구현하기

In [1]:
#pytorch 패키지
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary #model summary

#경고문구 무시
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
#AlexNet class
#'Imagenet' data의 class가 1000개 이므로 default: num_classes=1000
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet,self).__init__()
        #Convolution Layers
        self.features = nn.Sequential(
            #convolution layer 1
            nn.Conv2d(in_channels=3, out_channels=96,kernel_size=11,stride=4, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),

            #convolution layer 2
            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),

            #convolution layer 3
            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),

            #convolution layer 4
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),

            #convolution layer 5
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        #Fully-connected Layers
        self.classifier = nn.Sequential(
            nn.Linear(256*6*6, 4096),
            nn.ReLU(),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes)
        )

    def forward(self,x):
        x = self.features(x) #Convolution
        x = torch.flatten(x, 1) #1차원 벡터로
        x = self.classifier(x) #MLP
        return F.log_softmax(x) #softmax 출력

In [3]:
Net = AlexNet() #AlexNet 인스턴스 생성
summary(Net, (3, 227, 227)) #모델 요약

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
         MaxPool2d-3           [-1, 96, 27, 27]               0
            Conv2d-4          [-1, 256, 27, 27]         614,656
              ReLU-5          [-1, 256, 27, 27]               0
         MaxPool2d-6          [-1, 256, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         885,120
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]       1,327,488
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         884,992
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
           Linear-14                 [-

### 모델 개요
 - 합성곱층 (3,227,277) -> (256, 6, 6)
 - 1차원 벡터로 변환
 - FC층 (1,9216) -> (1,1000)
### Parameter
 - 활성화층과 풀링층에서 parameter가 0 개인것을 확인할 수 있다
 - 과제1(pdf참고해서 직접 계산한값)과 같은 결과를 보여준다

# AlexNet 고찰?

In [4]:
from torchvision import models

torchNet = models.AlexNet()
summary(torchNet, (3, 227, 227))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]          23,296
              ReLU-2           [-1, 64, 56, 56]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

In [5]:
torchNet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

## 직접 구현한 AlexNet과 pytorch에서 구현되있는 AlexNet에 차이가 있음을 확인할 수 있다
 - 차이점1: 합성곱층에서 channel의 수
 - 차이점2: AdaptiveAvgPooing
### 차이점 1
 - 원래의 AlexNet은 2개의 GPU로 모델을 병렬처리해서 학습을 진행한다
 - pytorch에서 구현한 AlexNet은 합성곱층은 데이터를 배치로 나눠서 n개의 GPU로 학습을 하고, 이후 FC층에서 모델을 병렬처리해서 n개의 GPU로 학습을 한다
 - 즉, 모델을 병렬처리하는 원래 모델 vs 데이터를 병렬 처리해서 학습 후 FC층에서 모델을 병렬 처리해서 학습 -> multi gpu
 - 참조1) pytorch github: https://github.com/pytorch/vision/pull/463#issuecomment-379214941
 - 참조2) Alex Krizhevsky(AlexNet만든 사람)의 multi-gpu사용 논문 One weird trick for parallelizing convolutional neural networks: https://arxiv.org/pdf/1404.5997.pdf
### 차이점 2
 - AdaptiveAveragePooling을 이용해 GAP(Global Average Pooling)의 효과를 얻으려 한것으로 예상된다 (정확한 정보 아닙니다)
 - 1차원 벡터로 변환될 때 손실되는 정보를 보완하기 위해 이용하는 것으로 예상된다
 - 참조1) pytorch,tensorflow에서 GAP: https://underflow101.tistory.com/41
 - 참조2) GAP&CAM이란? :https://poddeeplearning.readthedocs.io/ko/latest/CNN/CAM%20-%20Class%20Activation%20Map/